In [1]:
import pandas as pd

In [4]:
nox = pd.read_csv('Taiwan_nox_emissions.csv')
plant = pd.read_csv('Taiwan_powerplants.csv')

In [9]:
plant.shape

(11, 5)

In [10]:
nox.shape

(15500, 6)

In [6]:
plant.head()

,facility_id,data_source,name,iso2,geom
0,S3200688,tw_epa,台灣電力股份有限公司興達發電廠。,TW,POINT(120.202939764967 22.8508886719243)
1,F1700736,tw_epa,台灣電力股份有限公司林口發電廠。,TW,POINT(121.299694383972 25.1221981283711)
2,H4810851,tw_epa,華亞汽電股份有限公司華亞汽電廠。,TW,POINT(121.36482786733 25.0364911318493)
3,C1400170,tw_epa,台灣電力股份有限公司協和發電廠。,TW,POINT(121.739240933003 25.1569279229638)
4,E5400878,tw_epa,台灣電力股份有限公司南部發電廠。,TW,POINT(120.299374078243 22.6007508877674)


In [17]:
nox.head(10)

,facility_id,datetime,value,data_source,poll,unit
0,C1400170,2019-03-01,2833.708452,tw_epa,nox,kg
1,C1400170,2019-03-02,2838.442698,tw_epa,nox,kg
2,C1400170,2019-03-03,3248.871142,tw_epa,nox,kg
3,C1400170,2019-03-04,7802.961915,tw_epa,nox,kg
4,C1400170,2019-03-05,8781.186092,tw_epa,nox,kg
5,C1400170,2019-03-06,8475.531241,tw_epa,nox,kg
6,C1400170,2019-03-07,4334.106760,tw_epa,nox,kg
7,C1400170,2019-03-08,6649.995696,tw_epa,nox,kg
8,C1400170,2019-03-09,2985.525120,tw_epa,nox,kg
9,C1400170,2019-03-10,3417.604867,tw_epa,nox,kg


In [16]:
nox[nox['facility_id']== 'C1400170'].sort_values(by='datetime')

,facility_id,datetime,value,data_source,poll,unit
0,C1400170,2019-03-01,2833.708452,tw_epa,nox,kg
1,C1400170,2019-03-02,2838.442698,tw_epa,nox,kg
2,C1400170,2019-03-03,3248.871142,tw_epa,nox,kg
3,C1400170,2019-03-04,7802.961915,tw_epa,nox,kg
4,C1400170,2019-03-05,8781.186092,tw_epa,nox,kg
...,...,...,...,...,...,...
1552,C1400170,2023-09-26,4833.342712,tw_epa,nox,kg
1553,C1400170,2023-09-27,5003.181407,tw_epa,nox,kg
1554,C1400170,2023-09-28,6007.419708,tw_epa,nox,kg
1555,C1400170,2023-09-29,3614.927075,tw_epa,nox,kg


In [12]:
len(nox.datetime.unique())

1606

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb

max_depth = 4
min_child_weight = 5
subsample = 0.7
colsample_bytree = 0.7
objective = "reg:squarederror" #'reg:linear',#"reg:squarederror"
num_estimators = 500 #2000 #1000 #3000  #200
learning_rate =  0.1 #0.01  #0.05 #0.003 # 0.3

xgb_reg = xgb.XGBRegressor(max_depth=max_depth,
            min_child_weight=min_child_weight,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            objective=objective,
            n_estimators=num_estimators,
            learning_rate=learning_rate,
            early_stopping_rounds=100,
            num_boost_round = 2000)

kf = KFold(n_splits=4, random_state=42, shuffle=True) # n_splits was 5


i=0
testscore=[]
#feature_imp=pd.DataFrame({'feature': X.columns})
for train_index, test_index in kf.split(X, y):
#for train_index, test_index in gkf.split(X, y, groups):
#    X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:] # for unscaled dataframe
    X_train, X_test = X[train_index,:], X[test_index,:] # for scaled version
    y_train, y_test = y[train_index], y[test_index]
                      
    xgb_reg.fit(X_train, np.ravel(y_train)) 

    y_pred = xgb_reg.predict(X_test)
    test_score1 = mean_squared_error(y_test, y_pred)  
    
    testscore.append(test_score1)
    i=i+1
    feature_imp['importance'+str(i)]=xgb_reg.feature_importances_

feature_imp['mean']=feature_imp.iloc[:,1:i].mean(axis=1)
feature_imp['std']=feature_imp.iloc[:,1:i].std(axis=1)    
print('mean_squared_error on test_set:', testscore, np.mean(testscore))

In [18]:
data

,facility_id,datetime,value,data_source,poll,unit
0,C1400170,2019-03-01,2833.708452,tw_epa,nox,kg
1,C1400170,2019-03-02,2838.442698,tw_epa,nox,kg
2,C1400170,2019-03-03,3248.871142,tw_epa,nox,kg
3,C1400170,2019-03-04,7802.961915,tw_epa,nox,kg
4,C1400170,2019-03-05,8781.186092,tw_epa,nox,kg
...,...,...,...,...,...,...
15495,X0501232,2023-09-01,1052.145476,tw_epa,nox,kg
15496,X0501232,2023-09-02,959.300111,tw_epa,nox,kg
15497,X0501232,2023-09-03,1109.682234,tw_epa,nox,kg
15498,X0501232,2023-09-04,594.060077,tw_epa,nox,kg
